In [1]:
import os
from tqdm import tqdm 
import pandas as pd

# La funzione

In [3]:
def compute_stops(ais):
    ais=ais[['SOG','IMO','LATITUDE','LONGITUDE']].copy()

    # Crea la colonna 'is_stop' basata sulla velocità
    ais.loc[:, 'is_stop'] = ais['SOG'] < 3

    # Inizializza una lista per memorizzare i dati degli stop
    stop_data = []
    current_stop = None
    count = 0

    # Itera attraverso il DataFrame per identificare e contare gli stop consecutivi
    for index, row in ais.iterrows():
        if row['is_stop']:
            count += 1
            if current_stop is None:
                current_stop = row.copy()  # Inizia un nuovo stop
        else:
            if current_stop is not None:
                # Aggiungi il blocco di stop al DataFrame
                current_stop['DURATION'] = count
                stop_data.append(current_stop)
                current_stop = None
                count = 0

    # Aggiungi l'ultimo stop se il DataFrame termina con uno stop
    if current_stop is not None:
        current_stop['DURATION'] = count
        stop_data.append(current_stop)

    # Crea un DataFrame dai risultati
    stops = pd.DataFrame(stop_data)
    stops=stops[['IMO','LONGITUDE','LATITUDE','DURATION']]
    return stops


# Applichiamo la funzione a tutti i dataset

In [5]:
# Specifica la cartella di interesse
folder_path = '/kaggle/input/ais-tesi'

# Elenco di tutti i file nella cartella
files = os.listdir(folder_path)

# Stampa i nomi dei file
!mkdir /kaggle/working/output
for file in tqdm(files):
    ais=pd.read_csv('/kaggle/input/ais-tesi/'+file, low_memory=False)
    stops=compute_stops(ais)
    stops.to_csv('/kaggle/working/output/stops_'+str(ais['IMO'][0])+'.csv')

mkdir: cannot create directory '/kaggle/working/output': File exists


100%|██████████| 126/126 [02:07<00:00,  1.01s/it]


# Concateniamo tutti gli stops

In [10]:
cartella_csv = '/kaggle/working/output/'
file_csv = [f for f in os.listdir(cartella_csv) if f.endswith('.csv')]

dataframes = []

for file in tqdm(file_csv):
    percorso_file = os.path.join(cartella_csv, file)
    df = pd.read_csv(percorso_file)
    dataframes.append(df)

df_concatenato = pd.concat(dataframes, ignore_index=True)
df_concatenato.to_csv('/kaggle/working/output/stops.csv', index=False)


100%|██████████| 126/126 [00:00<00:00, 504.70it/s]
